In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import pandas as pd
import string
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import csv

In [ ]:
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                         options = options)

In [ ]:
xl_path = "/Users/pumpedup_py/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/GIT OMSA/Spring 2023/CSE 6242/Project/ListofCity.xlsx"
wb = openpyxl.load_workbook(xl_path)
main_sheet = wb["Sheet1"]

city_column_list = []

for i in range(1, main_sheet.max_row+1):
    city_name = main_sheet.cell(row=i, column=1).value
    city_column_list.append(city_name)

In [ ]:
xl_path = "/Users/pumpedup_py/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/GIT OMSA/Spring 2023/CSE 6242/Project/Population.xlsx"
wb = openpyxl.load_workbook(xl_path)
main_sheet = wb["population"]

for i in range(len(city_column_list)):
    main_sheet.cell(row=1, column=i+2).value = city_column_list[i]

wb.save(xl_path)

In [ ]:
# extracts table data from a web page, and cleans the data by removing specific characters.
# The cleaned population table data is returned.

def get_table_data():
    html_content = driver.page_source

    soup = BeautifulSoup(html_content, 'html.parser')

    div=soup.select_one("div#qfcontent")
    table=pd.read_html(str(div))
    table_data = table[1]["Unnamed: 1"].to_frame()

    spec_chars = [" "]
    for char in spec_chars:
        table_data = table_data.apply(lambda x: x.str.replace(char,''))
    
    return table_data

In [ ]:
def write_df_to_excel(file_path, sheet_name, df, column_idx):
    # Load the Excel file
    book = load_workbook(file_path)

    # Select the sheet you want to write to
    writer = pd.ExcelWriter(filename, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    ws = writer.sheets[sheet_name]

    # Load your data into a DataFrame
    df = df

    # Define the starting column index and row number to write to
    start_row = 2

    # Write the DataFrame to the Excel file
    for index, row in df.iterrows():
        for i, value in enumerate(row):
            column_index = column_idx + i
            column_letter = get_column_letter(column_index)
            cell = f'{column_letter}{start_row + index}'
            ws[cell] = value

    # Save the changes to the Excel file
    writer.save()

In [ ]:
# loops over a list of cities and for each city, 
# opens a webpage on the US Census Bureau website to extract population data for that city,
# and then writes the data to an Excel file.

for i in enumerate(city_list):
    driver.get("https://www.census.gov/quickfacts/fact/table/" + i[1] +"/PST045222")
    
    file_path = "/Users/pumpedup_py/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/GIT OMSA/Spring 2023/CSE 6242/Project/Population.xlsx"
    sheet_name = 'population'
    df = get_table_data()
    column_idx = i[0] + 2
    
    write_df_to_excel(file_path, sheet_name, df, column_idx)